In [1]:
import pandas as pd
import string
import nltk
import math
import tensorflow as tfl
import numpy as np
import operator
import regex as re
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tag import UnigramTagger
from nltk.corpus import treebank
from nltk.corpus import sentiwordnet as swn
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

pd.set_option('display.max_columns', None)

In [39]:
file = pd.read_csv("scrapping_BTC_comments.csv")
file

,created_utc,body,score,permalink,date
0,1.511280e+09,So /u/spez how much longer are you going to ig...,2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,Wow this is huge. I'm surprised /r/bitcoin mod...,631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,Holy shit....,720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"I'm a bot, *bleep*, *bloop*. Someone has linke...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,/u/spez I once shied away from replying to you...,34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,I don't know whether I should feel enraged by ...,212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,Great detective work!\r\n\r\n$10 /u/tippr,127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"Wow, epic, standing ovation! This confirms wha...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,This comment from /u/4n4n4 got the double dash...,23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,If I remember correctly Gregory Maxwell often ...,115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


In [3]:
len_file = len(file)
len_file

167

## Data Cleaning

In [4]:
file["body"]

0      So /u/spez how much longer are you going to ig...
1      Wow this is huge. I'm surprised /r/bitcoin mod...
2                                          Holy shit....
3      I'm a bot, *bleep*, *bloop*. Someone has linke...
4      /u/spez I once shied away from replying to you...
5      I don't know whether I should feel enraged by ...
6              Great detective work!\r\n\r\n$10 /u/tippr
7      Wow, epic, standing ovation! This confirms wha...
8      This comment from /u/4n4n4 got the double dash...
9      If I remember correctly Gregory Maxwell often ...
10     Paging u/adam3us, what do you feel about a C-l...
11     Smoke everywhere.  /u/bashco clearly had knowl...
12     The technical term is "false flag".\r\n\r\n>  ...
13                                               TL;DR? 
14                       Initiate Operation Dragonslayer
15     Thank you for doing this. More evidence to col...
16     The most telling aspect of this is the solid l...
17     I don't know who this ce

### Remove all "/u/", "/r/", "/r", "\n"

In [5]:
for i in range(len_file):
    file["body"][i] = re.sub(r"(\/\w\/|\w\/)", "", file["body"][i])
    file["body"][i] = re.sub(r"\\n?t?", "", file["body"][i])
    file["body"][i] = re.sub(r"[0-9]*", "", file["body"][i])

file.head(20)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,created_utc,body,score,permalink,date
0,1.511280e+09,So spez how much longer are you going to ignor...,2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,Wow this is huge. I'm surprised bitcoin mods w...,631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,Holy shit....,720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"I'm a bot, *bleep*, *bloop*. Someone has linke...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,spez I once shied away from replying to you re...,34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,I don't know whether I should feel enraged by ...,212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,Great detective work!\r\n\r\n$ tippr,127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"Wow, epic, standing ovation! This confirms wha...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"This comment from nn got the double dash, no s...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,If I remember correctly Gregory Maxwell often ...,115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


### Remove punctuation

In [6]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [7]:
file['body'] = file['body'].apply(lambda x: remove_punctuation(x))
file

,created_utc,body,score,permalink,date
0,1.511280e+09,So spez how much longer are you going to ignor...,2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,Wow this is huge Im surprised bitcoin mods wer...,631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,Holy shit,720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,Im a bot bleep bloop Someone has linked to thi...,244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,spez I once shied away from replying to you re...,34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,I dont know whether I should feel enraged by w...,212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,Great detective work\r\n\r\n tippr,127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,Wow epic standing ovation This confirms what w...,164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,This comment from nn got the double dash no sp...,23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,If I remember correctly Gregory Maxwell often ...,115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


### Remove short comments (in terms of words)

In [8]:
tokenizer = RegexpTokenizer(r'\w+')

In [9]:
file['body'] = file['body'].apply(lambda x : tokenizer.tokenize(x.lower()))
file.head(20)

,created_utc,body,score,permalink,date
0,1.511280e+09,"[so, spez, how, much, longer, are, you, going,...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, this, is, huge, im, surprised, bitcoin, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,"[holy, shit]",720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"[im, a, bot, bleep, bloop, someone, has, linke...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, i, once, shied, away, from, replying, t...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[i, dont, know, whether, i, should, feel, enra...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, this, confirms,...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[this, comment, from, nn, got, the, double, da...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[if, i, remember, correctly, gregory, maxwell,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


In [10]:
file = file[file['body'].map(len) > 2]
file

,created_utc,body,score,permalink,date
0,1.511280e+09,"[so, spez, how, much, longer, are, you, going,...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, this, is, huge, im, surprised, bitcoin, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
3,1.511279e+09,"[im, a, bot, bleep, bloop, someone, has, linke...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, i, once, shied, away, from, replying, t...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[i, dont, know, whether, i, should, feel, enra...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, this, confirms,...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[this, comment, from, nn, got, the, double, da...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[if, i, remember, correctly, gregory, maxwell,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34
10,1.511297e+09,"[paging, adamus, what, do, you, feel, about, a...",22,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 21:46:16


### Remove small words

In [11]:
for i in range(len_file):
    try :
        for word in file['body'][i]:
            if len(word) < 3:
                file['body'][i].remove(word)
    except KeyError :
        continue
        
file

,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, how, much, longer, are, you, going, ign...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, this, huge, surprised, bitcoin, mods, we...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
3,1.511279e+09,"[a, bot, bleep, bloop, someone, has, linked, t...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, once, shied, away, from, replying, you,...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, should, feel, enraged, w...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, this, confirms,...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[this, comment, from, got, the, double, dash, ...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[remember, correctly, gregory, maxwell, often,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34
10,1.511297e+09,"[paging, adamus, what, you, feel, about, cleve...",22,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 21:46:16


### Remove Stopwords

In [12]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [13]:
file['body'] = file['body'].apply(lambda x : remove_stopwords(x))
file

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, much, longer, going, ignore, blatant, c...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, huge, surprised, bitcoin, mods, sloppy, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
3,1.511279e+09,"[bot, bleep, bloop, someone, linked, thread, a...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, shied, away, replying, reddit, admins, ...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, feel, enraged, happened,...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, confirms, suspe...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[comment, got, double, dash, space, talk, usin...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[remember, correctly, gregory, maxwell, often,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34
10,1.511297e+09,"[paging, adamus, feel, clevel, executive, comp...",22,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 21:46:16


### Remove Reddit links

In [14]:
def remove_links(text):
    words = [w for w in text if not len(w) > 20]
    return words

In [15]:
file['body'] = file['body'].apply(lambda x : remove_links(x))
file

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, much, longer, going, ignore, blatant, c...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, huge, surprised, bitcoin, mods, sloppy, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
3,1.511279e+09,"[bot, bleep, bloop, someone, linked, thread, a...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, shied, away, replying, reddit, admins, ...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, feel, enraged, happened,...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, confirms, suspe...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[comment, got, double, dash, space, talk, usin...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[remember, correctly, gregory, maxwell, often,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34
10,1.511297e+09,"[paging, adamus, feel, clevel, executive, comp...",22,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 21:46:16


### Lemmatization

In [16]:
train_sents = treebank.tagged_sents()[:3000]
tagger = UnigramTagger(train_sents)

In [17]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemmatizedText = []
    words = nltk.word_tokenize(text)
    wordTokens = nltk.pos_tag(words)
    for wordToken in wordTokens:
        wordType = wordToken[1]
        word =  wordToken[0]
        
        
        if wordType[0:2] == "JJ":
            lemmatizedText.append(lemmatizer.lemmatize(word, "a"))
        elif wordType[0] == "V":
            lemmatizedText.append(lemmatizer.lemmatize(word, "v"))
        elif wordType[0] == "N":
            lemmatizedText.append(lemmatizer.lemmatize(word, "n"))
        elif wordType[0:2] == "RB":
            lemmatizedText.append(lemmatizer.lemmatize(word, "r"))
        else :
            lemmatizedText.append(lemmatizer.lemmatize(word))            
            
    return " ".join(lemmatizedText)

In [18]:
for i in range(len_file):
    try :
        for word in file["body"][i]:
            word = lemmatize(word)
    except KeyError :
        continue
        
file

,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, much, longer, going, ignore, blatant, c...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, huge, surprised, bitcoin, mods, sloppy, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
3,1.511279e+09,"[bot, bleep, bloop, someone, linked, thread, a...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, shied, away, replying, reddit, admins, ...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, feel, enraged, happened,...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, confirms, suspe...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[comment, got, double, dash, space, talk, usin...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[remember, correctly, gregory, maxwell, often,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34
10,1.511297e+09,"[paging, adamus, feel, clevel, executive, comp...",22,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 21:46:16


### Stemming

In [19]:
porter = PorterStemmer()

In [20]:
def stem_sentence(sentence):
    porter = PorterStemmer()
    for i in range(len(sentence)):
        word = porter.stem(sentence[i])
        sentence[i] = word
    return sentence

In [21]:
for i in range(len_file):
    try:
        file["body"][i] = stem_sentence(file["body"][i])
    except KeyError:
        continue
file

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, much, longer, go, ignor, blatant, censo...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, huge, surpris, bitcoin, mod, sloppi, abl...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
3,1.511279e+09,"[bot, bleep, bloop, someon, link, thread, anot...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, shi, away, repli, reddit, admin, ask, p...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, feel, enrag, happen, ast...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detect, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, stand, ovat, confirm, suspect, alo...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[comment, got, doubl, dash, space, talk, use, ...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[rememb, correctli, gregori, maxwel, often, re...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34
10,1.511297e+09,"[page, adamu, feel, clevel, execut, compani, m...",22,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 21:46:16


## Data analysis

### Unique words

In [22]:
uniqueWords = []

for sentence in file["body"]:
    words = [word for word in sentence]
    for word in words:
        if word not in uniqueWords:
            uniqueWords.append(word)

uniqueWords

['spez',
 'much',
 'longer',
 'go',
 'ignor',
 'blatant',
 'censorship',
 'manipul',
 'bitcoin',
 'mod',
 'wow',
 'huge',
 'surpris',
 'sloppi',
 'abl',
 'uncov',
 'vote',
 'bot',
 'mean',
 'involv',
 'hack',
 'wonder',
 'actual',
 'what',
 'crazi',
 'connect',
 'nullc',
 'show',
 'clearli',
 'blockstream',
 'work',
 'togeth',
 'entir',
 'market',
 'dumbfound',
 'would',
 'realli',
 'dont',
 'see',
 'reddit',
 'admin',
 'take',
 'action',
 'user',
 'account',
 'edit',
 'highli',
 'recommend',
 'messag',
 'look',
 'bleep',
 'bloop',
 'someon',
 'link',
 'thread',
 'anoth',
 'place',
 'againstastroturf',
 'evid',
 'rbitcoin',
 'may',
 'attack',
 'bcore',
 'berniesand',
 'bgold',
 'inform',
 'bitcoinexpos',
 'bitcoinuncensor',
 'bitcoincash',
 'bitcoindiscuss',
 'gmaxwel',
 'bitcoinmtl',
 'bitcoinno',
 'bitcoinxt',
 'cto',
 'greg',
 'maxwel',
 'caught',
 'use',
 'sock',
 'puppet',
 'scheme',
 'btcgreec',
 'buttcoin',
 'butteri',
 'casualconvers',
 'conspiraci',
 'crypto',
 'cryptocurr',
 

In [23]:
numOfWords = dict.fromkeys(uniqueWords, 0)
numOfWords

{'spez': 0,
 'much': 0,
 'longer': 0,
 'go': 0,
 'ignor': 0,
 'blatant': 0,
 'censorship': 0,
 'manipul': 0,
 'bitcoin': 0,
 'mod': 0,
 'wow': 0,
 'huge': 0,
 'surpris': 0,
 'sloppi': 0,
 'abl': 0,
 'uncov': 0,
 'vote': 0,
 'bot': 0,
 'mean': 0,
 'involv': 0,
 'hack': 0,
 'wonder': 0,
 'actual': 0,
 'what': 0,
 'crazi': 0,
 'connect': 0,
 'nullc': 0,
 'show': 0,
 'clearli': 0,
 'blockstream': 0,
 'work': 0,
 'togeth': 0,
 'entir': 0,
 'market': 0,
 'dumbfound': 0,
 'would': 0,
 'realli': 0,
 'dont': 0,
 'see': 0,
 'reddit': 0,
 'admin': 0,
 'take': 0,
 'action': 0,
 'user': 0,
 'account': 0,
 'edit': 0,
 'highli': 0,
 'recommend': 0,
 'messag': 0,
 'look': 0,
 'bleep': 0,
 'bloop': 0,
 'someon': 0,
 'link': 0,
 'thread': 0,
 'anoth': 0,
 'place': 0,
 'againstastroturf': 0,
 'evid': 0,
 'rbitcoin': 0,
 'may': 0,
 'attack': 0,
 'bcore': 0,
 'berniesand': 0,
 'bgold': 0,
 'inform': 0,
 'bitcoinexpos': 0,
 'bitcoinuncensor': 0,
 'bitcoincash': 0,
 'bitcoindiscuss': 0,
 'gmaxwel': 0,
 'bitc

In [24]:
for i in range(len_file):
    try :
        for word in file["body"][i]:
            numOfWords[word] += 1
    except KeyError:
        i += 1
        
numOfWords

{'spez': 5,
 'much': 11,
 'longer': 2,
 'go': 12,
 'ignor': 1,
 'blatant': 2,
 'censorship': 10,
 'manipul': 47,
 'bitcoin': 68,
 'mod': 47,
 'wow': 6,
 'huge': 1,
 'surpris': 8,
 'sloppi': 1,
 'abl': 4,
 'uncov': 2,
 'vote': 48,
 'bot': 22,
 'mean': 5,
 'involv': 37,
 'hack': 40,
 'wonder': 3,
 'actual': 7,
 'what': 6,
 'crazi': 1,
 'connect': 1,
 'nullc': 11,
 'show': 5,
 'clearli': 5,
 'blockstream': 17,
 'work': 24,
 'togeth': 2,
 'entir': 4,
 'market': 2,
 'dumbfound': 1,
 'would': 16,
 'realli': 7,
 'dont': 13,
 'see': 15,
 'reddit': 26,
 'admin': 8,
 'take': 8,
 'action': 2,
 'user': 3,
 'account': 11,
 'edit': 2,
 'highli': 3,
 'recommend': 1,
 'messag': 2,
 'look': 9,
 'bleep': 1,
 'bloop': 1,
 'someon': 10,
 'link': 7,
 'thread': 11,
 'anoth': 5,
 'place': 3,
 'againstastroturf': 1,
 'evid': 38,
 'rbitcoin': 50,
 'may': 31,
 'attack': 43,
 'bcore': 1,
 'berniesand': 1,
 'bgold': 1,
 'inform': 4,
 'bitcoinexpos': 1,
 'bitcoinuncensor': 1,
 'bitcoincash': 1,
 'bitcoindiscuss': 

### Term Frequency (TF)

In [25]:
def computeTF(file, uniqueWords):
    tf = []
    index = 0
    
    try :
        for sentence in file["body"]:
            tf.append(dict.fromkeys(uniqueWords, 0))
            for word in sentence:
                tf[index][word] += 1 / len(sentence)
            index += 1
    except KeyError :
        print("error")
    return tf

In [26]:
tf = computeTF(file, uniqueWords)
tf[0]

{'spez': 0.1,
 'much': 0.1,
 'longer': 0.1,
 'go': 0.1,
 'ignor': 0.1,
 'blatant': 0.1,
 'censorship': 0.1,
 'manipul': 0.1,
 'bitcoin': 0.1,
 'mod': 0.1,
 'wow': 0,
 'huge': 0,
 'surpris': 0,
 'sloppi': 0,
 'abl': 0,
 'uncov': 0,
 'vote': 0,
 'bot': 0,
 'mean': 0,
 'involv': 0,
 'hack': 0,
 'wonder': 0,
 'actual': 0,
 'what': 0,
 'crazi': 0,
 'connect': 0,
 'nullc': 0,
 'show': 0,
 'clearli': 0,
 'blockstream': 0,
 'work': 0,
 'togeth': 0,
 'entir': 0,
 'market': 0,
 'dumbfound': 0,
 'would': 0,
 'realli': 0,
 'dont': 0,
 'see': 0,
 'reddit': 0,
 'admin': 0,
 'take': 0,
 'action': 0,
 'user': 0,
 'account': 0,
 'edit': 0,
 'highli': 0,
 'recommend': 0,
 'messag': 0,
 'look': 0,
 'bleep': 0,
 'bloop': 0,
 'someon': 0,
 'link': 0,
 'thread': 0,
 'anoth': 0,
 'place': 0,
 'againstastroturf': 0,
 'evid': 0,
 'rbitcoin': 0,
 'may': 0,
 'attack': 0,
 'bcore': 0,
 'berniesand': 0,
 'bgold': 0,
 'inform': 0,
 'bitcoinexpos': 0,
 'bitcoinuncensor': 0,
 'bitcoincash': 0,
 'bitcoindiscuss': 0,
 

### Inverse Data Frequency (IDF)

In [27]:
idfs = dict.fromkeys(uniqueWords, 0)
idfs

{'spez': 0,
 'much': 0,
 'longer': 0,
 'go': 0,
 'ignor': 0,
 'blatant': 0,
 'censorship': 0,
 'manipul': 0,
 'bitcoin': 0,
 'mod': 0,
 'wow': 0,
 'huge': 0,
 'surpris': 0,
 'sloppi': 0,
 'abl': 0,
 'uncov': 0,
 'vote': 0,
 'bot': 0,
 'mean': 0,
 'involv': 0,
 'hack': 0,
 'wonder': 0,
 'actual': 0,
 'what': 0,
 'crazi': 0,
 'connect': 0,
 'nullc': 0,
 'show': 0,
 'clearli': 0,
 'blockstream': 0,
 'work': 0,
 'togeth': 0,
 'entir': 0,
 'market': 0,
 'dumbfound': 0,
 'would': 0,
 'realli': 0,
 'dont': 0,
 'see': 0,
 'reddit': 0,
 'admin': 0,
 'take': 0,
 'action': 0,
 'user': 0,
 'account': 0,
 'edit': 0,
 'highli': 0,
 'recommend': 0,
 'messag': 0,
 'look': 0,
 'bleep': 0,
 'bloop': 0,
 'someon': 0,
 'link': 0,
 'thread': 0,
 'anoth': 0,
 'place': 0,
 'againstastroturf': 0,
 'evid': 0,
 'rbitcoin': 0,
 'may': 0,
 'attack': 0,
 'bcore': 0,
 'berniesand': 0,
 'bgold': 0,
 'inform': 0,
 'bitcoinexpos': 0,
 'bitcoinuncensor': 0,
 'bitcoincash': 0,
 'bitcoindiscuss': 0,
 'gmaxwel': 0,
 'bitc

In [28]:
def computeIDF(idfs, numOfWords, file, len_file):
    for x, y in idfs.items():
        count = 0
        for i in range(len_file):
            try:
                for word in file["body"][i]:
                    if x == word:
                        count += 1
                        break
            except KeyError:
                i += 1
        if count != 0:
            idfs[x] = math.log(len(file) / count)
    return idfs

In [29]:
idfs = computeIDF(idfs, numOfWords, file, len_file)
idfs

{'spez': 3.258096538021482,
 'much': 2.670309873119363,
 'longer': 4.174387269895637,
 'go': 2.670309873119363,
 'ignor': 4.867534450455582,
 'blatant': 4.174387269895637,
 'censorship': 3.481240089335692,
 'manipul': 2.5649493574615367,
 'bitcoin': 1.4017985476558559,
 'mod': 2.469639177657212,
 'wow': 3.0757749812275277,
 'huge': 4.867534450455582,
 'surpris': 2.921624301400269,
 'sloppi': 4.867534450455582,
 'abl': 3.7689221617874726,
 'uncov': 4.174387269895637,
 'vote': 3.258096538021482,
 'bot': 2.3826278006675823,
 'mean': 3.258096538021482,
 'involv': 3.7689221617874726,
 'hack': 3.0757749812275277,
 'wonder': 3.7689221617874726,
 'actual': 2.921624301400269,
 'what': 3.258096538021482,
 'crazi': 4.867534450455582,
 'connect': 4.867534450455582,
 'nullc': 2.469639177657212,
 'show': 3.258096538021482,
 'clearli': 3.481240089335692,
 'blockstream': 2.302585092994046,
 'work': 1.8230120127321594,
 'togeth': 4.174387269895637,
 'entir': 3.7689221617874726,
 'market': 4.17438726989

### TF-IDF

In [30]:
tf[0]['spez']

0.1

In [31]:
def computeTF_IDF(tf, idfs):
    tab = []
    for i in range(len(tf)):
        dico = {}
        for mot in tf[i]:
            dico[mot] = tf[i][mot] * idfs[mot]
        tab.append(dico)
    tfidf = pd.DataFrame(tab)
    return tfidf     

In [32]:
tfidf = computeTF_IDF(tf, idfs)
tfidf

abl  absenc  absolut   account     accus    action     activ  \
0    0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
1    0.0739     0.0      0.0  0.052359  0.000000  0.081851  0.000000   
2    0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
3    0.0000     0.0      0.0  0.054496  0.000000  0.000000  0.000000   
4    0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
5    0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
6    0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.120997   
7    0.0000     0.0      0.0  0.046040  0.000000  0.000000  0.000000   
8    0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
9    0.0000     0.0      0.0  0.000000  0.097079  0.000000  0.097079   
10   0.0000     0.0      0.0  0.140543  0.000000  0.000000  0.000000   
11   0.0000     0.0      0.0  0.000000  0.000000  0.219705  0.000000   
12   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
13   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
14   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
15   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
16   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
17   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
18   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
19   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
20   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
21   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
22   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
23   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
24   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
25   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
26   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
27   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
28   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
29   0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
..      ...     ...      ...       ...       ...       ...       ...   
100  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
101  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
102  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
103  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
104  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
105  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
106  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
107  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
108  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
109  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
110  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
111  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
112  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
113  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
114  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
115  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
116  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
117  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
118  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
119  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
120  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
121  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
122  0.0000     0.0      0.0  0.000000  0.000000  0.000000  0.000000   
123  0.0000     0.0      0.0  0.000000  0.000000

### Get important words

In [33]:
columns = tfidf.columns.values
words_importance = {}
index = 0
for label in columns:
    words_importance[label] = sum(tfidf[columns[index]])
    index += 1
    
words_importance = sorted(words_importance.items(), key=operator.itemgetter(1), reverse = True)[0:50]
words_importance

[('bitcoin', 4.895406897811347),
 ('tippr', 4.575270055160287),
 ('delet', 4.061365513865612),
 ('post', 3.900925818960958),
 ('work', 3.7235590033065558),
 ('someon', 3.6651138285771396),
 ('rbitcoin', 3.5169633523130925),
 ('damn', 3.5038687726437208),
 ('thank', 3.370808307064116),
 ('reddit', 3.3113931495110442),
 ('pleas', 3.220753055319337),
 ('gild', 3.1627742437380686),
 ('surpris', 2.960015664374077),
 ('great', 2.8454462095918256),
 ('time', 2.8381360180821074),
 ('sticki', 2.6494792603802546),
 ('know', 2.4478187818811747),
 ('dr', 2.433767225227791),
 ('peep', 2.433767225227791),
 ('pin', 2.433767225227791),
 ('pl', 2.433767225227791),
 ('upvotedownvot', 2.433767225227791),
 ('analysi', 2.385364154226078),
 ('big', 2.385364154226078),
 ('gold', 2.337404059982536),
 ('mod', 2.289955567393453),
 ('like', 2.2725774513530954),
 ('get', 2.2341758300092596),
 ('fuck', 2.1485699892654555),
 ('btc', 2.145221962299348),
 ('dude', 2.0871936349478184),
 ('tell', 2.0149185955696916),
 

### Get important documents

In [34]:
tfidf_transposed = tfidf.T
tfidf_transposed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129
abl,0.0,0.073900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.062296,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.005617,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
absenc,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.007254,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
absolut,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.243377,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
account,0.0,0.052359,0.000000,0.054496,0.000000,0.000000,0.000000,0.046040,0.000000,0.000000,0.140543,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.190736,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.022069,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.007959,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.090519,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.070271
accus,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.097079,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,

In [35]:
columns = tfidf_transposed.columns.values
important_documents = {}
index = 0
for document in columns:
    important_documents[document] = sum(tfidf_transposed[columns[index]])
    index += 1
    
important_documents = sorted(important_documents.items(), key=operator.itemgetter(1), reverse = True)[0:50]
important_documents

[(126, 4.867534450455582),
 (12, 4.636485390268933),
 (76, 4.423643366584094),
 (57, 4.381056913191754),
 (121, 4.370553120497982),
 (113, 4.347674065035623),
 (59, 4.257740268893291),
 (119, 4.255493483084523),
 (110, 4.208174344555559),
 (109, 4.1872949741231515),
 (92, 4.177011996056994),
 (74, 4.174387269895637),
 (65, 4.162323773309448),
 (55, 4.131295066221488),
 (9, 4.091148198642458),
 (124, 4.0905118247392025),
 (52, 4.087179245934879),
 (91, 4.062815494238532),
 (101, 4.0537341300245195),
 (84, 4.034056680824808),
 (46, 4.027887885999201),
 (102, 4.009167789897847),
 (82, 4.00055606202432),
 (122, 3.9908927825403824),
 (120, 3.9865913733629332),
 (75, 3.98495243649156),
 (79, 3.9671087603201953),
 (89, 3.955670435138478),
 (32, 3.9286572669445166),
 (99, 3.928216304989104),
 (45, 3.9185617278535156),
 (41, 3.9046494669090293),
 (22, 3.8964544693236745),
 (47, 3.8945793759279255),
 (81, 3.8945793759279255),
 (27, 3.8743739775117807),
 (7, 3.8664793182981647),
 (67, 3.861733135

## Making a function to automize the process

In [36]:
tab_files = os.listdir("../Data/Reddit_Data/data_history/V3_more_attributes/btc/comments")
tab_files

['2015-12-15.csv',
 '2015-12-16.csv',
 '2015-12-17.csv',
 '2015-12-30.csv',
 '2015-12-31.csv',
 '2016-01-01.csv',
 '2016-01-18.csv',
 '2016-01-19.csv',
 '2016-01-20.csv',
 '2016-01-21.csv',
 '2016-01-25.csv',
 '2016-01-26.csv',
 '2016-01-27.csv',
 '2016-02-01.csv',
 '2016-02-08.csv',
 '2016-02-11.csv',
 '2016-02-12.csv',
 '2016-02-14.csv',
 '2016-02-25.csv',
 '2016-02-26.csv',
 '2016-02-27.csv',
 '2016-02-28.csv',
 '2016-02-29.csv',
 '2016-03-01.csv',
 '2016-03-02.csv',
 '2016-03-03.csv',
 '2016-03-04.csv',
 '2016-03-05.csv',
 '2016-03-06.csv',
 '2016-03-12.csv',
 '2016-03-13.csv',
 '2016-03-14.csv',
 '2016-03-15.csv',
 '2016-03-18.csv',
 '2016-05-02.csv',
 '2016-05-03.csv',
 '2016-05-04.csv',
 '2016-05-05.csv',
 '2016-05-06.csv',
 '2016-05-07.csv',
 '2016-05-08.csv',
 '2016-06-30.csv',
 '2016-07-01.csv',
 '2016-07-02.csv',
 '2016-07-03.csv',
 '2016-07-13.csv',
 '2016-07-26.csv',
 '2016-09-22.csv',
 '2016-11-14.csv',
 '2016-11-15.csv',
 '2016-11-16.csv',
 '2016-11-17.csv',
 '2016-11-18

In [37]:
def tf_idf_function(path, file_path):
    file = pd.read_csv(file_path.append("/").append(path))
    len_file = len(file)
    
    #Data Cleaning
    for i in range(len_file):
        file["body"][i] = re.sub(r"(\/\w\/|\w\/)", "", file["body"][i])
        file["body"][i] = re.sub(r"\\n?t?", "", file["body"][i])
        file["body"][i] = re.sub(r"[0-9]*", "", file["body"][i])
    
    file['body'] = file['body'].apply(lambda x: remove_punctuation(x))
    tokenizer = RegexpTokenizer(r'\w+')
    file['body'] = file['body'].apply(lambda x : tokenizer.tokenize(x.lower()))
    file = file[file['body'].map(len) > 2]
    for i in range(len_file):
        try :
            for word in file['body'][i]:
                if len(word) < 3:
                    file['body'][i].remove(word)
        except KeyError :
            continue
            
    file['body'] = file['body'].apply(lambda x : remove_stopwords(x))
    file['body'] = file['body'].apply(lambda x : remove_links(x))
    train_sents = treebank.tagged_sents()[:3000]
    tagger = UnigramTagger(train_sents)
    for i in range(len_file):
        try :
            for word in file["body"][i]:
                word = lemmatize(word)
        except KeyError :
            continue
    porter = PorterStemmer()
    for i in range(len_file):
        try:
            file["body"][i] = stem_sentence(file["body"][i])
        except KeyError:
            continue
    
    #Data Analysis
    uniqueWords = []

    for sentence in file["body"]:
        words = [word for word in sentence]
        for word in words:
            if word not in uniqueWords:
                uniqueWords.append(word)
                
    numOfWords = dict.fromkeys(uniqueWords, 0)
    for i in range(len_file):
        try :
            for word in file["body"][i]:
                numOfWords[word] += 1
        except KeyError:
            i += 1
            
    tf = computeTF(file, uniqueWords)
    
    idfs = dict.fromkeys(uniqueWords, 0)
    idfs = computeIDF(idfs, numOfWords, file, len_file)
    
    tfidf = computeTF_IDF(tf, idfs)
    
    #Get important words
    columns = tfidf.columns.values
    words_importance = {}
    index = 0
    for label in columns:
        words_importance[label] = sum(tfidf[columns[index]])
        index += 1
    
    words_importance = sorted(words_importance.items(), key=operator.itemgetter(1), reverse = True)[0:50]
    
    #Get important documents
    tfidf_transposed = tfidf.T
    columns = tfidf_transposed.columns.values
    important_documents = {}
    index = 0
    for document in columns:
        important_documents[document] = sum(tfidf_transposed[columns[index]])
        index += 1
    
    important_documents = sorted(important_documents.items(), key=operator.itemgetter(1), reverse = True)[0:50]